# 1. Peridically fetch traffic trends and attack spikes from clodfare and/or IPS from abuse IPDP
# 2. Classify IPS with high DDos confidence score using machine learning
# 3, Remote IPs to coordinate and show them in fromtend globe

In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

In [2]:
import requests 

API_TOKEN = "kvVDk6bPlZ8WE305dH5EqtcMTG8tf0RkCvzJIeqL"
url="https://api.cloudflare.com/client/v4/user/tokens/verify"

headers={
    "Authorization": f"Bearer {API_TOKEN}"
}

response=requests.get(url=url,headers=headers)
# print(response.status_code)
# print(response.json())

In [30]:
API_TOKEN = "kvVDk6bPlZ8WE305dH5EqtcMTG8tf0RkCvzJIeqL"
HEADERS = {"Authorization": f"Bearer {API_TOKEN}"}

PARAMS = {
    "aggInterval": "1h",
    "dateRange": "4w",
    "format": "json"
}

def fetch_data(url):
    response = requests.get(url, headers=HEADERS, params=PARAMS)
    if response.status_code == 200:
        return response.json().get("result", {})
    else:
        print(f"Error {response.status_code}: {response.text}")
        return []

# Layer 3 URL
vector_url = "https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/vector"
ip_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/ip_version"
protocol_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/protocol"
bitrate_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/bitrate"
duration_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/duration"

vector_data = fetch_data(vector_url)
ip_data=fetch_data(ip_url)
protocol_data=fetch_data(protocol_url)
bitrate_data=fetch_data(bitrate_url)
duration_data=fetch_data(duration_url)


df_vector = pd.DataFrame(vector_data)
df_ip = pd.DataFrame(ip_data)
df_protocol = pd.DataFrame(protocol_data)
df_bitrate = pd.DataFrame(bitrate_data)
df_duration = pd.DataFrame(duration_data)


In [31]:
to_wide=[vector_data,ip_data,protocol_data,bitrate_data,duration_data]
names=["vector_data","ip_data","protocol_data","bitrate_data","duration_data"]

In [32]:
dfs={}
for name,data in zip(names,to_wide):
    data=data["serie_0"]
    dfs[name]=pd.DataFrame(data)    

df_vector=dfs["vector_data"]
df_duration=dfs["duration_data"]
df_ip=dfs["ip_data"]
df_protocol=dfs["protocol_data"]
df_bitrate=dfs["bitrate_data"]

In [34]:
for df in [df_vector, df_protocol, df_bitrate, df_duration, df_ip]:
    df['timestamps'] = pd.to_datetime(df['timestamps'], utc=True)
df_ip

,timestamps,IPv4,IPv6
0,2025-07-19 16:00:00+00:00,99.71089,0.28911
1,2025-07-19 17:00:00+00:00,99.466463,0.533537
2,2025-07-19 18:00:00+00:00,99.556629,0.443371
3,2025-07-19 19:00:00+00:00,99.492527,0.507473
4,2025-07-19 20:00:00+00:00,99.639625,0.360375
...,...,...,...
668,2025-08-16 12:00:00+00:00,99.28446,0.71554
669,2025-08-16 13:00:00+00:00,99.170567,0.829433
670,2025-08-16 14:00:00+00:00,99.854295,0.145705
671,2025-08-16 15:00:00+00:00,99.669227,0.330773


In [7]:
# data=vector_data["serie_0"]
# df=pd.DataFrame(data)
# # df_vector=df.melt(
# #     id_vars=["timestamps"],
# #     var_name="attack_type",
# #     value_name="value"
# # )

In [8]:
# data=ip_data["serie_0"]
# df=pd.DataFrame(data)
# df_ip=df.melt(
#     id_vars=["timestamps"],
#     var_name="version"
# )

In [9]:
# data=protocol_data["serie_0"]
# df=pd.DataFrame(data)
# df_protocol=df.melt(
#     id_vars=["timestamps"],
#     var_name="protocol"
# )

In [10]:
# data=bitrate_data["serie_0"]
# df=pd.DataFrame(data)
# df_bitrate=df.melt(
#     id_vars=["timestamps"],
#     var_name="bitrate"
# )


In [11]:
# data=duration_data["serie_0"]
# df=pd.DataFrame(data)
# df_duration=df.melt(
#     id_vars=["timestamps"],
#     var_name="duration"
# )


In [12]:
# df_duration
# df_bitrate
# df_protocol
# # df_ip
# # df_vector

In [13]:
# for df in [df_duration, df_bitrate, df_protocol, df_ip, df_vector]:
#     df['timestamps'] = pd.to_datetime(df['timestamps'], errors='coerce')

In [14]:
# for df in [df_vector, df_protocol, df_bitrate, df_duration, df_ip]:
#     df['timestamps'] = pd.to_datetime(df['timestamps'], utc=True)

In [15]:
# df_vector_hourly = df_vector.groupby(['attack_type', pd.Grouper(key='timestamps', freq='1H')])['value'].agg(['sum', 'mean', 'max']).reset_index()

In [16]:
# df